In [1]:
import psycopg2 # need install
import json
from annotation import Annotator

In [2]:
# loading from config file
with open("config.json", "r") as file:
    config = json.load(file)


In [3]:
db_config = config["db"]
conn = psycopg2.connect(
            host=db_config["host"],
            dbname=db_config["dbname"],
            user=db_config["user"],
            password=db_config["pwd"]
    )
if conn:
    print("Connected to database %s sucessfully!" % db_config["dbname"])
else:
    print("Connection failed.")

Connected to database db4031 sucessfully!


```python
#query with template
query_temp = "SELECT * FROM nation WHERE nation.n_regionkey = %s;"
vals = (1,)
query = cursor.mogrify(query_temp, vals)
```

```python
# Explain params?
cursor.execute("EXPLAIN (analyze, buffers, verbose, format json) " + query)
cursor.execute("EXPLAIN (FORMAT JSON) " + query)
```

In [4]:
if conn:
    cursor = conn.cursor()
    query = "SELECT * FROM customer c, orders o WHERE c.c_custkey = o.o_custkey ORDER BY c.c_custkey"
    cursor.execute("EXPLAIN (FORMAT JSON)" + query)
    plans = cursor.fetchall()

In [5]:
plans

[([{'Plan': {'Node Type': 'Merge Join',
     'Parallel Aware': False,
     'Async Capable': False,
     'Join Type': 'Inner',
     'Startup Cost': 166134.47,
     'Total Cost': 367414.5,
     'Plan Rows': 1500000,
     'Plan Width': 266,
     'Inner Unique': True,
     'Merge Cond': '(o.o_custkey = c.c_custkey)',
     'Plans': [{'Node Type': 'Gather Merge',
       'Parent Relationship': 'Outer',
       'Parallel Aware': False,
       'Async Capable': False,
       'Startup Cost': 166116.2,
       'Total Cost': 340815.92,
       'Plan Rows': 1500000,
       'Plan Width': 107,
       'Workers Planned': 2,
       'Plans': [{'Node Type': 'Sort',
         'Parent Relationship': 'Outer',
         'Parallel Aware': False,
         'Async Capable': False,
         'Startup Cost': 165116.18,
         'Total Cost': 166678.68,
         'Plan Rows': 625000,
         'Plan Width': 107,
         'Sort Key': ['o.o_custkey'],
         'Plans': [{'Node Type': 'Seq Scan',
           'Parent Relationship

In [6]:
x = Annotator()

In [7]:
print(x.wrapper(plans))

Step 1: Perform sequential scan on table orders as o. 
Step 2: Perform a sort on table orders with sort key ['o.o_custkey']. 
Step 3: The results of the previous operation are gathered and merged. 
Step 4: Perform index scan on table customer as c using index on customer_pkey. 
Step 5: Perform a merge join on tables orders and customer under the condition (o.o_custkey = c.c_custkey). 
 to get the final result.
